In [38]:
import pandas as pd 

In [39]:
df = pd.read_csv('data/data.csv', encoding='latin1')

In [40]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [42]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [43]:
df.shape

(541909, 8)

## Data Cleaning

In [44]:
# remove alphabets from the invoice column
df = df[~df['InvoiceNo'].str.contains('C', na=False)]

In [45]:
df = df.dropna(subset=['CustomerID'])

In [46]:
df = df[df['Quantity'] > 0]

In [47]:
df = df[df['UnitPrice'] > 0]

In [48]:
# Keep only numeric stock codes
df = df[df['StockCode'].str.isnumeric()]

In [49]:
df['StockCode'].unique()

array(['71053', '22752', '21730', ..., '23561', '90089', '23843'],
      shape=(2785,), dtype=object)

In [50]:
# Remove rows with no description
df = df[df['Description'].notna()]

In [56]:
#datatime format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [51]:
# Add Total Price
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

In [57]:
# Revenue per customer
revenue_per_customer = df.groupby('CustomerID')['TotalPrice'].sum().reset_index(name='Revenue')
revenue_per_customer

,CustomerID,Revenue
0,12346.0,77183.60
1,12347.0,3653.45
2,12348.0,1437.24
3,12349.0,1372.42
4,12350.0,258.00
...,...,...
4309,18280.0,180.60
4310,18281.0,80.82
4311,18282.0,178.05
4312,18283.0,1964.26


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 363087 entries, 1 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    363087 non-null  object 
 1   StockCode    363087 non-null  object 
 2   Description  363087 non-null  object 
 3   Quantity     363087 non-null  int64  
 4   InvoiceDate  363087 non-null  object 
 5   UnitPrice    363087 non-null  float64
 6   CustomerID   363087 non-null  float64
 7   Country      363087 non-null  object 
 8   TotalPrice   363087 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 27.7+ MB


In [54]:
# df.isna().sum()

In [ ]:
# RFM Metrics
snapshot_date = df_clean['InvoiceDate'].max() + pd.Timedelta(days=1)

rfm = df_clean.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',                                  # Frequency
    'TotalPrice': 'sum'                                      # Monetary
}).reset_index()

rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalPrice': 'Monetary'
}, inplace=True)

# Show summary results
revenue_per_customer.head(), rfm.head()

In [58]:
# RFM Metrics after cleaning [Recency, Frequency, Monetary]
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,    
    'InvoiceNo': 'nunique',                                     
    'TotalPrice': 'sum'                                         
}).reset_index()
rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalPrice': 'Monetary'
}, inplace=True)

In [59]:
rfm.head()

,CustomerID,Recency,Frequency,Monetary
0,12346.0,326,1,77183.60
1,12347.0,2,7,3653.45
2,12348.0,75,4,1437.24
3,12349.0,19,1,1372.42
4,12350.0,310,1,258.00
